In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)
tf.enable_eager_execution()

from projects.feed.rank.src import config
from projects.feed.rank.src.model import Model
from projects.feed.rank.src.tfrecord_dataset import Dataset

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
product='sgsapp'
mark='tuwen'

In [4]:
# 载入模型
model_hour = '2020050819'
valid_hour = '2020050822'
# valid_hour = '2020030812'
model_root = f'/search/odin/publicData/CloudS/libowei/rank_online/data/{mark}_hour_sgsapp_v1/models/8'
# model_root = '/home/gezi/tmp/rank/exps/tuwen/v29/dlrm-norm/'
model_dir = f'{model_root}/{model_hour}'
# model_dir = model_root
tfrecord_root = f'/search/odin/publicData/CloudS/libowei/rank/{product}/data/{mark}_hour_{product}_v1/tfrecords'
tfrecord_dir = f'{tfrecord_root}/{valid_hour}'
print('model:', model_dir)
print('record:', tfrecord_dir)
argv = open(f'{model_dir}/command.txt').readline().strip().replace('data_version=2', 'data_version=1').split()
FLAGS(argv)
FLAGS.eager = True
# FLAGS.fields_norm=True
config.init()
model = Model()
melt.eager.restore(model, model_dir)

model: /search/odin/publicData/CloudS/libowei/rank_online/data/tuwen_hour_sgsapp_v1/models/8/2020050819
record: /search/odin/publicData/CloudS/libowei/rank/newmse/data/tuwen_hour_newmse_v1/tfrecords/2020050822


2020-05-12 14:03:06 0:00:00 Tf dataset and Tf model train in Eager mode, keras False, distributed:False
2020-05-12 14:03:06 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-05-12 14:03:06 0:00:00 batch_size: 512 eval_batch_size: 512 batch_size_per_gpu: 512 num_gpus: 0 gpu: [] CUDA_VISIABLE_DEVICES=[-1] work_mode: train distributed: False horovod: False
2020-05-12 14:03:06 0:00:00 model: [8] model_dir: [/search/odin/libowei/rank/tuwen/model/8]
2020-05-12 14:03:07 0:00:00 Latest checkpoint: /search/odin/publicData/CloudS/libowei/rank_online/data/tuwen_hour_sgsapp_v1/models/8/2020050819/model.ckpt-2020050819-1.00-9677


In [5]:
dataset = Dataset('valid')

In [6]:
files = gezi.list_files(tfrecord_dir)
total = melt.get_num_records(files) 
total

4162592

In [8]:
batch_size = 512
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False, shuffle=True)
num_steps = -int(-total // batch_size)

In [9]:
xs = []
for i, (x, _) in tqdm(enumerate(batches), total=num_steps, ascii=True, desc='loop'):
  model(x)
  break

loop:   0%|          | 0/8131 [00:00<?, ?it/s]

In [12]:
model.deep.x_all

<tf.Tensor: id=874, shape=(512, 94, 32), dtype=float32, numpy=
array([[[ 5.6850e-01,  1.3466e-01,  1.6126e-01, ...,  2.0559e-01,
          3.0121e-01,  2.8068e-01],
        [-1.7477e-02, -8.8006e-03, -3.8504e-02, ...,  2.0034e-02,
         -2.1419e-02,  8.4295e-02],
        [-9.4730e-01,  1.9611e+00,  3.1698e+00, ...,  1.4435e+00,
         -1.3255e+00,  1.2816e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]],

       [[-6.0657e-02, -1.9167e-01, -1.2993e-01, ..., -4.6300e-02,
         -4.4084e-01,  3.0533e-02],
        [-2.2878e-01,  9.1265e-02, -8.6723e-02, ..., -1.9738e-01,
          4.4939e-03,  7.8245e-02],
        [-3.2795e+00,  2.3117e+00,  2.8664e+00, ...,  1.3823e+00,
         -3.4593e-01,  1.1049e+00],
    

In [32]:
# 这两个doc 第一个位于位置1 未点击            《楞严经》未得谓得未证言证 
# 第二个位于位置11 点击  时长276              不管你相不相信转世和业，修持金刚萨埵法门非常重要
x1 = get_doc(xs, '2041980l3wSzO6')
x2 = get_doc(xs, '20506m0l326Btq')

In [54]:
model(x1)

<tf.Tensor: id=438689, shape=(1, 1), dtype=float32, numpy=array([[0.037]], dtype=float32)>

In [55]:
model.d

<tf.Tensor: id=438589, shape=(1, 1), dtype=float32, numpy=array([[-0.8849]], dtype=float32)>

In [56]:
model.w

<tf.Tensor: id=438372, shape=(1, 1), dtype=float32, numpy=array([[1.0864]], dtype=float32)>

In [57]:
model.prob_click

<tf.Tensor: id=438630, shape=(1, 1), dtype=float32, numpy=array([[0.2072]], dtype=float32)>

In [58]:
model.y_click

<tf.Tensor: id=438599, shape=(1, 1), dtype=float32, numpy=array([[-1.3418]], dtype=float32)>

In [64]:
gezi.sigmoid(0.0054 * 1.0864 + 0.0344 * (-0.8849) + (-1.3172))

0.20721847628049556

In [65]:
x_all1 = model.deep.x_all

In [66]:
x_all1

<tf.Tensor: id=438554, shape=(1, 94, 32), dtype=float32, numpy=
array([[[ 0.459 ,  0.0914, -0.2821, ...,  0.3804, -0.4798, -1.0662],
        [-0.0323,  0.0285,  0.019 , ..., -0.0047, -0.0163,  0.0139],
        [ 1.8274,  3.1585, -0.1991, ..., -1.6778, -0.8073, -2.459 ],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]]],
      dtype=float32)>

In [80]:
model(x2)

<tf.Tensor: id=439766, shape=(1, 1), dtype=float32, numpy=array([[0.0022]], dtype=float32)>

In [81]:
model.d

<tf.Tensor: id=439666, shape=(1, 1), dtype=float32, numpy=array([[-50.9848]], dtype=float32)>

In [82]:
model.w

<tf.Tensor: id=439449, shape=(1, 1), dtype=float32, numpy=array([[-1.2384]], dtype=float32)>

In [83]:
model.prob_click

<tf.Tensor: id=439707, shape=(1, 1), dtype=float32, numpy=array([[0.044]], dtype=float32)>

In [84]:
x_all2 = model.deep.x_all

In [71]:
x_all2

<tf.Tensor: id=438932, shape=(1, 94, 32), dtype=float32, numpy=
array([[[ 0.459 ,  0.0914, -0.2821, ...,  0.3804, -0.4798, -1.0662],
        [-0.0391,  0.0165,  0.0103, ...,  0.022 ,  0.0049, -0.0378],
        [ 1.8274,  3.1585, -0.1991, ..., -1.6778, -0.8073, -2.459 ],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]]],
      dtype=float32)>

In [74]:
x1['x_all'] = x_all2
model(x1)
# 可以看到wide部分几乎没有影响到最终得分

<tf.Tensor: id=439411, shape=(1, 1), dtype=float32, numpy=array([[0.0022]], dtype=float32)>

In [75]:
model.d

<tf.Tensor: id=439311, shape=(1, 1), dtype=float32, numpy=array([[-50.9848]], dtype=float32)>

In [76]:
model.w

<tf.Tensor: id=439276, shape=(1, 1), dtype=float32, numpy=array([[1.0864]], dtype=float32)>

In [78]:
model.prob_click

<tf.Tensor: id=439352, shape=(1, 1), dtype=float32, numpy=array([[0.0445]], dtype=float32)>

In [79]:
# 保持wide不变 更好 deep  的fields x_all 点击改了只是稍稍高了一点点 几乎可以忽略
gezi.sigmoid(0.0054 * 1.0864 + 0.0344 * (-50.9848) + (-1.3172))

0.04456531408785941

In [85]:
def rename(field):
  for x in maps:
#     print(x, len(x))
#     print('0', field, x[0], x[1])
    field = field.replace(x[0], x[1])
#     print('1', field, x[0], x[1])
  return field

In [130]:
other_fields = [x.name.split('/')[-2] for x in model.variables if x.name.endswith('embeddings:0')and not '/emb/emb/' in x.name]
other_fields[model.deep.history_index:model.deep.history_index] = model.deep.history_fields
onehot_fields = gezi.get_global('fields')
maps = [x.strip().split() for x in open('../conf/fields_map.txt')]
onehot_fields2 = [rename(x) for x in onehot_fields]
fields = other_fields + onehot_fields2
fields2 = other_fields + onehot_fields
# [(x, y) for x, y in zip(onehot_fields, onehot_fields2)]

In [110]:
import copy
x_all2_npy = x_all2.numpy()
x_all1_npy = x_all1.numpy()

In [114]:
x_all2_npy

array([[[ 0.459 ,  0.0914, -0.2821, ...,  0.3804, -0.4798, -1.0662],
        [-0.0391,  0.0165,  0.0103, ...,  0.022 ,  0.0049, -0.0378],
        [ 1.8274,  3.1585, -0.1991, ..., -1.6778, -0.8073, -2.459 ],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]]],
      dtype=float32)

In [115]:
x_all = x_all2_npy.copy()
x_all[0][1] = x_all1_npy[0][1]
x_all

array([[[ 0.459 ,  0.0914, -0.2821, ...,  0.3804, -0.4798, -1.0662],
        [-0.0323,  0.0285,  0.019 , ..., -0.0047, -0.0163,  0.0139],
        [ 1.8274,  3.1585, -0.1991, ..., -1.6778, -0.8073, -2.459 ],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]]],
      dtype=float32)

In [119]:
x2['x_all'] = x_all
model(x2)
model.prob_click

<tf.Tensor: id=441285, shape=(1, 1), dtype=float32, numpy=array([[0.0338]], dtype=float32)>

In [136]:
l = []
for i in tqdm(range(len(fields))):
  x_all = x_all1_npy.copy()
  x_all[0][i] = x_all2_npy[0][i]
  x1['x_all'] = x_all
  model(x1)
  prob_click1 = model.prob_click.numpy()[0][0]
  x_all = x_all2_npy.copy()
  x_all[0][i] = x_all1_npy[0][i]
  x2['x_all'] = x_all
  model(x2)
  prob_click2 = model.prob_click.numpy()[0][0]
  l.append((i, fields[i], fields2[i], prob_click1, prob_click2, prob_click2 - prob_click1))
  l.sort(key=lambda x: x[-1], reverse=True)
gezi.pprint(pd.DataFrame(l, columns=['fid', 'field', 'ori_field', 'pclick1', 'pclick2', 'delta']))

100%|██████████| 94/94 [00:03<00:00, 30.57it/s]


 +-------+------------------------------+-----------------+-----------+-----------+--------------+
|   fid | field                        | ori_field       |   pclick1 |   pclick2 |        delta |
|-------+------------------------------+-----------------+-----------+-----------+--------------|
|    64 | MInterestArtPornScoreKw      | MITATPORNSKW    |  0.142161 | 0.141803  | -0.000357732 |
|    63 | MInterestArtPornScoreAccount | MITATPORNSAC    |  0.154762 | 0.0963815 | -0.0583803   |
|    59 | MInterestArtPicCntAccount    | MITATPCNTAC     |  0.178602 | 0.0948361 | -0.0837662   |
|     5 | timespan_emb                 | timespan_emb    |  0.175528 | 0.0546406 | -0.120887    |
|    29 | IArtKw2                      | IATKWSE         |  0.174993 | 0.0520846 | -0.122908    |
|    71 | MInterestArtTag              | MITATTG         |  0.198853 | 0.0743374 | -0.124515    |
|    72 | MInterestArtTopic            | MITATTP         |  0.179713 | 0.0522657 | -0.127448    |
|    53 | MInteres